In [12]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [13]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe'

In [14]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [15]:
query = '풀무원 바른먹거리'

In [23]:
def biz_khan(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 #추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
    last_page =(int(page)-1)*10+1#추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
    domain = '한겨례'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1028&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        #url = url_domain + query + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('span', class_='title').text)
                head_list.append(head)
                tags = soup.select_one('#a-left-scroll-in > div.article-text > div').find_all('div', class_='text')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

biz = biz_khan(query,driver_path,3)

None_type
None_type


In [24]:
biz.to_csv('한겨례.csv',encoding='UTF8',index=None)

In [25]:
test = pd.read_csv('한겨례.csv',encoding='UTF8')

In [26]:
test

,언론사,제목,내용
0,한겨례,고기 없는 ‘바른 먹거리’ 꿈일까요,풀무원이 내놓는 식품의 정책과 방향을 만든 남기선 상무는 “풀무원 제품에 고기를 쓰...
1,한겨례,풀무원 ‘1호 사원’ 입사 34년만에 최고경영자 됐다,풀무원 남승우 대표이사 풀무원 제공 ‘바른 먹거리’를 표방하며 33년 동안 풀무원을...
2,한겨례,공익활동 위한 ‘풀무원 재단’ 출범,풀무원은 16일 사회적 책임 활동 강화를 위해 ‘풀무원 재단’을 만들었다 재단은 비...
3,한겨례,풀무원 새 발표…식품·친환경 의지 강조,올해 전문경영인 체제로 전환환 식품기업 풀무원이 새 기업이미지를 발표하고 계열사들의...
4,한겨례,풀무원홀딩스 ‘1조 클럽’ 입성,풀무원홀딩스가 지난해 연결 매출 1조1204억원을 기록해 식품 업계 ‘1조 클럽’에...
5,한겨례,4월 9일 쇼핑 단신,백화점서 무선인터넷 무료 이용 롯데백화점은 케이티와 손잡고 26개 모든 점포에 케이...
6,한겨례,아이밥상 따로 차려볼까 어린이 전용먹거리 ‘봇물’,다시다도 따로 간장도 따로 소금도 따로 김치도 따로… 저출산이 추세화하는 대신 하나...
7,한겨례,“비교하면 욕심 끝없어…나이 들어 안 물러나는 건 헛짓”,국회의원과 부천시장 등 30년 동안 7선의 선출직 공직생활을 지내고 은퇴한 원혜영 ...
8,한겨례,‘브랜드의 고향’이 관광상품으로,브랜드 한국에 새 숨결을 ⑨ 체험 마케팅의 숨은 가치 오스트리아 인스부르크에서 차로...
9,한겨례,“일 25씩 더 해 기부하면 세계 굶주림 사라져”,“세상의 아름다움 중에서 인간의 아름다움을 딛고 넘어설 만한 아름다움은 없다고 생각...
